## Content Recommender

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
df = pd.read_excel("/content/furniture dataset.xlsx")
df = df[["product id", "category", "color",	"material", "length",	"breadth",	"height",	"price"]]
df.head()

,product id,category,color,material,length,breadth,height,price
0,bb01,bean bag,darkslategray,velvet,175,167,100,5620
1,bs02,bookshelf,sienna,wood,200,62,501,12300
2,ca01,cabinet,dimgray,plastic,196,71,244,10000
3,ca02,cabinet,gainsboro,wood,115,34,122,15000
4,ch01,chair,darkslategray,leather,82,82,94,5500


In [ ]:
df = df.dropna()

In [ ]:
df_encoded = pd.get_dummies(df[["product id", "category",	"color",	"material"]], columns=["category",	"color",	"material"])
df_encoded.head()

,product id,category_bean bag,category_bookshelf,category_cabinet,category_chair,category_chair,category_desk,category_lamp,category_night stand,category_ottoman,...,material_ceramic,material_cloth,material_foam,material_glass,material_leather,material_metal,material_plastic,material_plastic,material_velvet,material_wood
0,bb01,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,bs02,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,ca01,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,ca02,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,ch01,0,0,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [ ]:
scaler = StandardScaler()
df[df['length']=="*"] = 220
scaled_data = scaler.fit_transform(df[["length",	"breadth",	"height",	"price"]])
df_scaled = pd.DataFrame(scaled_data, columns =["length",	"breadth",	"height",	"price"])
df_scaled.head()

,length,breadth,height,price
0,-0.168045,-0.176289,-0.187156,-0.736912
1,-0.167467,-0.183296,-0.155711,0.689461
2,-0.167560,-0.182695,-0.175864,0.198344
3,-0.169433,-0.185164,-0.185431,1.265989
4,-0.170196,-0.181961,-0.187627,-0.762535


In [ ]:
final_df = df_encoded.join(df_scaled)
final_df = final_df.dropna()
final_df.reset_index(drop=True, inplace=True)
final_df.head()

,product id,category_bean bag,category_bookshelf,category_cabinet,category_chair,category_chair,category_desk,category_lamp,category_night stand,category_ottoman,...,material_leather,material_metal,material_plastic,material_plastic,material_velvet,material_wood,length,breadth,height,price
0,bb01,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,-0.168045,-0.176289,-0.187156,-0.736912
1,bs02,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,-0.167467,-0.183296,-0.155711,0.689461
2,ca01,0,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,-0.167560,-0.182695,-0.175864,0.198344
3,ca02,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,-0.169433,-0.185164,-0.185431,1.265989
4,ch01,0,0,0,1,0,0,0,0,0,...,1,0,0,0,0,0,-0.170196,-0.181961,-0.187627,-0.762535


In [ ]:
similarity = cosine_similarity(final_df.iloc[:, 1:])

In [ ]:
def get_content_recommendations(id, wt, top_n=10):
    idx = final_df[final_df['product id'] == id].index[0]
    similarity_scores = { final_df['product id'][i]:similarity[idx][i]*wt for i in range(83) }
    similarity_scores = sorted(similarity_scores.items(), key=lambda x: x[1], reverse=True)
    return similarity_scores[1:6]

In [ ]:
# title = input("Enter Product Id: ")
print("Top 5 similar Furniture:")

# print(get_recommendations(title))
print(get_content_recommendations("so01", 0.3))

Top 5 similar Furniture:
[('so17', 0.2868855088050201), ('so15', 0.24743154985111657), ('ch18', 0.20306405198467978), ('so16', 0.195033772701252), ('so02', 0.17731474804089117)]


## Collaborative Recommender

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
from collections import Counter

In [ ]:
furniture_data = pd.read_excel("/content/furniture dataset.xlsx")
furniture_data.head()

,product id,product,product link,product name,category,color,material,support color,support material,product image,image link,length,breadth,height,price,description
0,bb01,bb01.glb,https://firebasestorage.googleapis.com/v0/b/cr...,NaN,bean bag,darkslategray,velvet,darkslategray,velvet,bb01.jpg,https://firebasestorage.googleapis.com/v0/b/cr...,175,167,100,5620,NaN
1,bs02,bs02.glb,https://firebasestorage.googleapis.com/v0/b/cr...,NaN,bookshelf,sienna,wood,peru,wood,bs02.jpg,https://firebasestorage.googleapis.com/v0/b/cr...,200,62,501,12300,NaN
2,ca01,ca01.glb,https://firebasestorage.googleapis.com/v0/b/cr...,NaN,cabinet,dimgray,plastic,dimgray,plastic,ca01.jpg,https://firebasestorage.googleapis.com/v0/b/cr...,196,71,244,10000,NaN
3,ca02,ca02.glb,https://firebasestorage.googleapis.com/v0/b/cr...,NaN,cabinet,gainsboro,wood,gainsboro,wood,ca02.jpg,https://firebasestorage.googleapis.com/v0/b/cr...,115,34,122,15000,NaN
4,ch01,ch01.gltf,https://firebasestorage.googleapis.com/v0/b/cr...,NaN,chair,darkslategray,leather,darkslategray,metal,ch01.jpg,https://firebasestorage.googleapis.com/v0/b/cr...,82,82,94,5500,NaN


In [ ]:
allFurniture = set(furniture_data['product id'])

In [ ]:
df1 = pd.read_csv("/content/Furniture preferences (Responses) - Form responses 1.csv")
df1 = df1.drop(['Timestamp', 'Email address'], axis=1)
df1.head()

,Age,Sex,No. of members in your home,Are you employed?,Earning member's monthly income,Check your preferences.,Check your preferences..1,Check your preference.,Check your preferences..2,Check your preferences..3
0,20,Female,5,no,0,"ch02, ch03, ch06, ch07, ch08, ch10, ch15, ch19...","ta04, ta10, ta14, ta20","so04, so05, so08","ca03, ca05, ca06, ca08, bs02, wa01, wa02","la02, la08, la09, la11"
1,46,Male,5,yes,100000,"ch06, ch08, ch10, ch15, ch27","ta07, ta11, ta14, ta18","so01, so03, so04, so05, so06, so09, so11, so17...","ca01, ca02, ca03, ca05, ca07","la02, la04, la08"
2,22,Female,6,yes,25000,"ch07, ch17","ta12, ta15","so10, so15, so17","ca03, wa01","la04, la06"
3,20,Female,6,no,800000,ch18,ta05,so03,ca06,la06
4,20,Male,4,no,0,"ch01, ch08, ch12, ch25","ta05, ta11, ta14","so06, so09, so14, so18","ca03, ca06, bs02, wa01, wa02","la03, la04"


In [ ]:
for i in allFurniture:
  df1[i] = [0]*25

In [ ]:
furr = []
for i in range(25):
  temp = (df1.iloc[i, 5]+", "+df1.iloc[i, 6]+", "+df1.iloc[i, 7]+", "+df1.iloc[i, 8]+", "+df1.iloc[i, 9]).split(", ")
  furr.append(temp)
  for j in temp:
    try:
      df1[j][i] = 1
    except KeyError:
      df1[j] = [0]*25
      df1[j][i] = 1

In [ ]:
furr = []
for i in range(25):
  furr.append((df1.iloc[i, 5]+", "+df1.iloc[i, 6]+", "+df1.iloc[i, 7]+", "+df1.iloc[i, 8]+", "+df1.iloc[i, 9]).split(", "))

In [ ]:
df1 = df1.drop(df1.columns[[5, 6, 7, 8, 9]], axis = 1)
df1.head(1)

,Age,Sex,No. of members in your home,Are you employed?,Earning member's monthly income,ch17,la08,ta05,ta03,to01,...,so06,ns01,ta10,ch27,ch11,ch12,po01,ca03,ch03,ch08
0,20,Female,5,no,0,0,1,0,0,0,...,0,0,1,0,0,0,0,1,1,1


In [ ]:
df_encoded1 = pd.get_dummies(df1[["Sex", 'Are you employed?']], drop_first=True)
df_encoded1.head()

,Sex_Male,Are you employed?_yes
0,0,0
1,1,1
2,0,1
3,0,0
4,1,0


In [ ]:
scaler1 = StandardScaler()
scaled_data1 = scaler1.fit_transform(df1[["Age ",	"No. of members in your home",	"Earning member's monthly income"]])
df_scaled1 = pd.DataFrame(scaled_data1, columns =["Age",	"No. of members in your home",	"Earning member's monthly income"])
df_scaled1.head()

,Age,No. of members in your home,Earning member's monthly income
0,-0.325692,0.567819,-0.890280
1,3.454656,0.567819,-0.559321
2,-0.034896,1.455036,-0.807540
3,-0.325692,1.455036,1.757392
4,-0.325692,-0.319398,-0.890280


In [ ]:
cf_df = df_encoded1.join(df_scaled1)
cf_df = cf_df.join(df1.iloc[:, 5:])
cf_df = cf_df.dropna()
cf_df

,Sex_Male,Are you employed?_yes,Age,No. of members in your home,Earning member's monthly income,ch17,la08,ta05,ta03,to01,...,so06,ns01,ta10,ch27,ch11,ch12,po01,ca03,ch03,ch08
0,0,0,-0.325692,0.567819,-0.890280,0,1,0,0,0,...,0,0,1,0,0,0,0,1,1,1
1,1,1,3.454656,0.567819,-0.559321,0,1,0,0,0,...,1,0,0,1,0,0,0,1,0,1
2,0,1,-0.034896,1.455036,-0.807540,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,0,0,-0.325692,1.455036,1.757392,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,-0.325692,-0.319398,-0.890280,0,0,1,0,0,...,1,0,0,0,0,1,0,1,0,1
5,0,0,-0.180294,-0.319398,2.419310,0,0,0,0,0,...,1,1,1,1,0,0,0,1,0,1
6,0,0,-0.325692,-0.319398,0.433556,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
7,1,1,-0.180294,-1.206615,-0.393841,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,1
8,0,1,-0.325692,-0.319398,-0.625512,1,0,0,0,0,...,1,0,1,0,0,0,0,0,1,1
9,0,0,-0.325692,3.229469,-0.857184,0,0,0,0,0,...,0,1,0,0,0,0,0,1,1,0


In [ ]:
similarity1 = cosine_similarity(cf_df)

In [ ]:
def (id, wt, top_n=5):
    similarity_scores1 = list(similarity[id])
    temp = pd.DataFrame()
    temp['user_id'] = [i for i in range(25)]
    temp['score'] = cosine_similarity(cf_df)[id]
    temp['wishlisted'] = furr
    temp = temp.sort_values(by=['score'], ascending=False)
    dic = {}
    for i in range(2, 6):
      for j in temp['wishlisted'][i]:
        if(j not in dic):
          dic[j] = []
        dic[j].append(temp['score'][i])
    final = []
    for key, value in dic.items():
      simi = np.mean(np.array(value))*wt
      final.append((key, simi))
    sorted_final = sorted(final, key = lambda x: x[1], reverse=True)

    return(sorted_final)

In [ ]:
# title = int(input("Enter User Id: "))
print("Top 10 similar Furniture:")
get_collaborative_recommendations(1, 0.7)

Top 10 similar Furniture:


[('ch06', 0.22003247371578874),
 ('ch09', 0.22003247371578874),
 ('ch10', 0.22003247371578874),
 ('ch15', 0.22003247371578874),
 ('ch21', 0.22003247371578874),
 ('ch27', 0.22003247371578874),
 ('ta02', 0.22003247371578874),
 ('ta04', 0.22003247371578874),
 ('ta10', 0.22003247371578874),
 ('so04', 0.22003247371578874),
 ('so05', 0.22003247371578874),
 ('so08', 0.22003247371578874),
 ('so11', 0.22003247371578874),
 ('so13', 0.22003247371578874),
 ('ca01', 0.22003247371578874),
 ('ns01', 0.22003247371578874),
 ('la01', 0.22003247371578874),
 ('la02', 0.22003247371578874),
 ('la10', 0.22003247371578874),
 ('ch01', 0.2106639208410088),
 ('ch08', 0.2106639208410088),
 ('so06', 0.2106639208410088),
 ('so18', 0.2106639208410088),
 ('ch12', 0.20129536796622882),
 ('ch25', 0.20129536796622882),
 ('ta11', 0.20129536796622882),
 ('ta14', 0.20129536796622882),
 ('so09', 0.20129536796622882),
 ('so14', 0.20129536796622882),
 ('bs02', 0.20129536796622882),
 ('wa02', 0.20129536796622882),
 ('la03', 0.

## Hybrid Recommender

In [ ]:
def hybrid(furniture, user_id):
  array = []
  for i in furniture:
    array += get_content_recommendations(i, 0.3)
    array += get_collaborative_recommendations(user_id, 0.6)
    sorted_array = sorted(array, key = lambda x: x[1], reverse=True)
  return sorted_array

hybrid(["so01", "so17"], 0)[:10]

[('so17', 0.2868855088050201),
 ('so01', 0.2868855088050201),
 ('so15', 0.2861679987751194),
 ('so15', 0.24743154985111657),
 ('so16', 0.21956001172425257),
 ('so11', 0.20721977670765904),
 ('ch18', 0.20306405198467978),
 ('ch06', 0.20122528056608366),
 ('ch09', 0.20122528056608366),
 ('ch10', 0.20122528056608366)]

In [ ]:
def get_recommendations(id, top_n=5):
    similarity_scores = list(enumerate(similarity[id]))
    temp = pd.DataFrame()
    temp['user_id'] = [i for i in range(25)]
    temp['score'] = similarity_scores
    temp.head()